In [1]:
import lmdb
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
env = lmdb.open('/mnt/h/datasets/csaw_seq_lv_set', map_size=1099511627776)

In [3]:
with env.begin() as txn:
   myList = list(txn.cursor().iternext(values=False))

In [6]:
len(myList)

6088

In [2]:
ann_data = pd.read_csv('anon_dataset_nonhidden_211125.csv')
ann_data

,anon_patientid,exam_year,anon_filename,x_age,x_case,x_cancer_laterality,x_type,x_lymphnode_met,rad_timing,rad_r1,rad_r2,rad_recall,rad_recall_type_right,rad_recall_type_left,imagelaterality,viewposition,libra_breastarea,libra_densearea,libra_percentdensity
0,2,2015,00002_20990909_L_CC_1.dcm,1,1,Left,3.0,0.0,2.0,0.0,0.0,0.0,NaN,NaN,Left,CC,127.258090,29.595217,23.256058
1,2,2015,00002_20990909_L_MLO_1.dcm,1,1,Left,3.0,0.0,2.0,0.0,0.0,0.0,NaN,NaN,Left,MLO,122.318120,39.298000,32.127705
2,2,2015,00002_20990909_R_CC_1.dcm,1,1,Left,3.0,0.0,2.0,0.0,0.0,0.0,NaN,NaN,Right,CC,114.570630,23.637600,20.631468
3,2,2015,00002_20990909_R_MLO_1.dcm,1,1,Left,3.0,0.0,2.0,0.0,0.0,0.0,NaN,NaN,Right,MLO,133.202380,36.162785,27.148750
4,4,2012,00004_20990909_R_CC_1.dcm,1,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,Right,CC,201.187730,68.998276,34.295467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98783,11301,2015,11301_20990909_R_CC_1.dcm,1,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,Right,CC,69.278275,24.969255,36.041973
98784,11303,2014,11303_20990909_R_CC_1.dcm,2,1,Left,3.0,0.0,1.0,1.0,1.0,1.0,NaN,2.0,Right,CC,196.125440,33.502670,17.082268
98785,11303,2014,11303_20990909_L_MLO_1.dcm,2,1,Left,3.0,0.0,1.0,1.0,1.0,1.0,NaN,2.0,Left,MLO,257.890530,15.262912,5.918369
98786,11303,2014,11303_20990909_R_MLO_1.dcm,2,1,Left,3.0,0.0,1.0,1.0,1.0,1.0,NaN,2.0,Right,MLO,232.718640,14.388752,6.182896


In [3]:
data_list = pd.read_csv('/mnt/h/datasets/csaw_seq_lv_set/seq_lv_val_set.csv')

In [4]:
new_data_list = []

In [5]:
import random
for idx, row in data_list.iterrows():
    id = int(row['id'].split('_')[0])
    p_row = ann_data[ann_data['anon_patientid']==id]
    x_cancer_laterality = p_row.iloc[0]['x_cancer_laterality']

    l_data = [
        row['id'], row['l_cc_view_0'], row['l_mlo_view_0'],row['l_cc_view_1'], row['l_mlo_view_1'],row['l_cc_view_2'], row['l_mlo_view_2'],row['rad_time']
    ]

    r_data = [
        row['id'], row['r_cc_view_0'], row['r_mlo_view_0'],row['r_cc_view_1'], row['r_mlo_view_1'],row['r_cc_view_2'], row['r_mlo_view_2'],row['rad_time']
    ]

    if x_cancer_laterality=='Left':
        new_data = l_data
    elif x_cancer_laterality=='Right':
        new_data = r_data
    else:
        new_data = random.choice([l_data, r_data])
    new_data_list.append(new_data)

In [6]:
df = pd.DataFrame(new_data_list,columns =['id','cc_view_0','mlo_view_0','cc_view_1','mlo_view_1','cc_view_2','mlo_view_2','rad_time'])
df.to_csv('/mnt/h/datasets/csaw_seq_lv_set/seq_lv_val_set_.csv', index=False)

In [3]:
train_set, test_set = train_test_split(data_list, test_size=0.1, random_state=32)
train_set, val_set = train_test_split(train_set, test_size=0.1, random_state=32)

In [4]:
print(len(train_set), len(val_set), len(test_set))

2957 329 366


In [5]:
train_set.to_csv('/mnt/c/Users/11351/Desktop/datasets/csaw_breast_lv_set/train_set.csv', index= False)
val_set.to_csv('/mnt/c/Users/11351/Desktop/datasets/csaw_breast_lv_set/val_set.csv', index= False)
test_set.to_csv('/mnt/c/Users/11351/Desktop/datasets/csaw_breast_lv_set/test_set.csv', index= False)